In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Parameters
batch_size = 128
image_size = 32
nc = 3  # Number of channels (CIFAR-10 is RGB)
nz = 100  # Size of latent vector
ngf = 64  # Size of feature maps in generator
ndf = 64  # Size of feature maps in discriminator
num_epochs = 100
lr = 0.0002
beta1 = 0.5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 8 x 8
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 16 x 16
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 32 x 32
        )

    def forward(self, input):
        return self.main(input)

# Discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 32 x 32
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 16 x 16
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 8 x 8
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)

# Initialize generator and discriminator
netG = Generator().to(device)
netD = Discriminator().to(device)

# Initialize weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

netG.apply(weights_init)
netD.apply(weights_init)

# Loss function and optimizers
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Fixed noise for visualization
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Training loop
def train_dcgan():
    G_losses = []
    D_losses = []
    
    print("Starting Training Loop...")
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader, 0):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
            netD.zero_grad()
            # Format batch
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), 1.0, device=device)
            # Forward pass real batch through D
            output = netD(real_cpu)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            errD_real.backward()
            D_x = output.mean().item()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            # Generate fake image batch with G
            fake = netG(noise)
            label.fill_(0.0)
            # Classify all fake batch with D
            output = netD(fake.detach())
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            # Add the gradients from the all-real and all-fake batches
            errD = errD_real + errD_fake
            # Update D
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.fill_(1.0)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = netD(fake)
            # Calculate G's loss based on this output
            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()
            # Update G
            optimizerG.step()
            
            # Save Losses for plotting later
            G_losses.append(errG.item())
            D_losses.append(errD.item())
            
            # Output training stats
            if i % 50 == 0:
                print(f'[{epoch}/{num_epochs}][{i}/{len(train_loader)}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f}/{D_G_z2:.4f}')
            
            # Check how the generator is doing by saving G's output on fixed_noise
            if (epoch == num_epochs-1) and (i == len(train_loader)-1):
                with torch.no_grad():
                    fake = netG(fixed_noise).detach().cpu()
                    save_image(fake, f'dcgan_result_epoch_{epoch}.png', normalize=True)

    return G_losses, D_losses

# Run training
G_losses, D_losses = train_dcgan()

# Plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="G")
plt.plot(D_losses, label="D")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig("dcgan_loss_plot.png")
plt.show()

# Save model checkpoints
torch.save(netG.state_dict(), "dcgan_generator.pth")
torch.save(netD.state_dict(), "dcgan_discriminator.pth")

# Generate and save a batch of fake images
def generate_fake_images(num_images=16):
    netG.eval()
    with torch.no_grad():
        noise = torch.randn(num_images, nz, 1, 1, device=device)
        fake_images = netG(noise).detach().cpu()
        
        # Denormalize the images
        fake_images = (fake_images + 1) / 2
        
        # Save the images
        grid = make_grid(fake_images, nrow=4)
        save_image(grid, "dcgan_generated_images.png")
        return grid

# Generate sample images
sample_images = generate_fake_images()

# Function to detect fake images using the discriminator
def detect_fake_images(images):
    netD.eval()
    with torch.no_grad():
        images = images.to(device)
        # D returns probability that each image is real
        scores = netD(images)
        predictions = (scores >= 0.5).float()  # 1 for real, 0 for fake
        return scores, predictions

print("DCGAN implementation complete!")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Parameters
batch_size = 64
image_size = 32
latent_dim = 512
n_mlp = 8  # Number of layers in mapping network
style_dim = 512
num_channels = 3  # RGB for CIFAR-10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 100
lr_g = 0.0001  # Reduced learning rate for generator
lr_d = 0.0002  # Original learning rate for discriminator
beta1 = 0.5
beta2 = 0.999

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Pixel normalization layer
class PixelNorm(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x / torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + 1e-8)

# Equalized learning rate layers with scaling factor
class EqualizedLinear(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        linear = nn.Linear(in_dim, out_dim)
        # Initialize weights with scaled variance
        scale = 1 / np.sqrt(in_dim)
        nn.init.normal_(linear.weight.data, 0.0, 1.0)
        linear.weight.data.mul_(scale)
        nn.init.zeros_(linear.bias.data)
        self.linear = linear
        self.scale = scale
        
    def forward(self, x):
        return self.linear(x)

class EqualizedConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # Initialize weights with scaled variance
        scale = 1 / np.sqrt(in_channels * kernel_size * kernel_size)
        nn.init.normal_(conv.weight.data, 0.0, 1.0)
        conv.weight.data.mul_(scale)
        nn.init.zeros_(conv.bias.data)
        self.conv = conv
        self.scale = scale
        
    def forward(self, x):
        return self.conv(x)

# Noise injection layer
class NoiseInjection(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.weight = nn.Parameter(torch.zeros(1, channels, 1, 1))
        
    def forward(self, x):
        batch, _, height, width = x.shape
        noise = torch.randn(batch, 1, height, width, device=x.device)
        return x + self.weight * noise

# Mapping Network
class MappingNetwork(nn.Module):
    def __init__(self, latent_dim, style_dim, n_mlp):
        super().__init__()
        layers = [PixelNorm()]
        for i in range(n_mlp):
            layers.append(EqualizedLinear(latent_dim if i == 0 else style_dim, style_dim))
            layers.append(nn.LeakyReLU(0.2))
        self.mapping = nn.Sequential(*layers)
        
    def forward(self, z):
        return self.mapping(z)

# Improved AdaIN
class AdaIN(nn.Module):
    def __init__(self, style_dim, channels):
        super().__init__()
        self.instance_norm = nn.InstanceNorm2d(channels)
        self.style_scale = EqualizedLinear(style_dim, channels)
        self.style_bias = EqualizedLinear(style_dim, channels)
        
    def forward(self, x, style):
        style = style.view(style.size(0), -1)
        scale = self.style_scale(style).unsqueeze(2).unsqueeze(3)
        bias = self.style_bias(style).unsqueeze(2).unsqueeze(3)
        
        # Apply instance normalization
        norm_x = self.instance_norm(x)
        
        # Apply style modulation
        return norm_x * (scale + 1) + bias  # Add 1 to scale to prevent vanishing

# Improved Style Block with noise injection
class StyleBlock(nn.Module):
    def __init__(self, in_channels, out_channels, style_dim, use_noise=True):
        super().__init__()
        self.use_noise = use_noise
        self.conv = EqualizedConv2d(in_channels, out_channels, 3, padding=1)
        if use_noise:
            self.noise = NoiseInjection(out_channels)
        self.adain = AdaIN(style_dim, out_channels)
        self.activation = nn.LeakyReLU(0.2)
        
    def forward(self, x, style):
        x = self.conv(x)
        if self.use_noise:
            x = self.noise(x)
        x = self.adain(x, style)
        return self.activation(x)

# ToRGB layer
class ToRGB(nn.Module):
    def __init__(self, in_channels, style_dim):
        super().__init__()
        self.conv = EqualizedConv2d(in_channels, 3, 1)
        self.adain = AdaIN(style_dim, 3)
        
    def forward(self, x, style):
        x = self.conv(x)
        x = self.adain(x, style)
        return x

# Improved Generator with more layers and style blocks
class StyleGANGenerator(nn.Module):
    def __init__(self, style_dim, n_mlp, channels=32):
        super().__init__()
        self.style_dim = style_dim
        
        # Mapping network
        self.mapping = MappingNetwork(latent_dim, style_dim, n_mlp)
        
        # Initial constant input
        self.constant_input = nn.Parameter(torch.randn(1, channels, 4, 4))
        
        # Style blocks - More layers for better generation
        self.style1 = StyleBlock(channels, channels, style_dim)
        self.style2 = StyleBlock(channels, channels * 2, style_dim)
        self.style3 = StyleBlock(channels * 2, channels * 4, style_dim)
        self.style4 = StyleBlock(channels * 4, channels * 2, style_dim)
        self.style5 = StyleBlock(channels * 2, channels, style_dim)
        
        # Upsampling layers
        self.upsample1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)  # 4x4 -> 8x8
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)  # 8x8 -> 16x16
        self.upsample3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)  # 16x16 -> 32x32
        
        # RGB conversion layers
        self.to_rgb1 = ToRGB(channels, style_dim)
        self.to_rgb2 = ToRGB(channels * 2, style_dim)
        self.to_rgb3 = ToRGB(channels * 4, style_dim)
        self.to_rgb4 = ToRGB(channels * 2, style_dim)
        self.to_rgb = ToRGB(channels, style_dim)
        
    def forward(self, z, return_latents=False, inject_index=None, truncation=None, truncation_latent=None, input_is_latent=False):
        # Map latent vector to style if not already a style vector
        if not input_is_latent:
            w = self.mapping(z)
        else:
            w = z
            
        # Apply truncation trick - helps stabilize training
        if truncation is not None and truncation_latent is not None:
            w = truncation_latent + truncation * (w - truncation_latent)
        
        # Start from constant input
        batch_size = z.size(0)
        x = self.constant_input.repeat(batch_size, 1, 1, 1)
        
        # Apply style blocks and upsampling with skip connections
        x = self.style1(x, w)
        
        x = self.upsample1(x)  # 4x4 -> 8x8
        x = self.style2(x, w)
        
        x = self.upsample2(x)  # 8x8 -> 16x16
        x = self.style3(x, w)
        
        x = self.style4(x, w)
        
        x = self.upsample3(x)  # 16x16 -> 32x32
        x = self.style5(x, w)
        
        # Final RGB conversion
        rgb = self.to_rgb(x, w)
        
        if return_latents:
            return torch.tanh(rgb), w
        else:
            return torch.tanh(rgb)  # Output range [-1, 1]

# Improved Discriminator with minibatch standard deviation
class MinibatchStdDev(nn.Module):
    def __init__(self, group_size=4):
        super().__init__()
        self.group_size = group_size
        
    def forward(self, x):
        batch_size, channels, height, width = x.shape
        
        # Ensure group size is not larger than batch size
        group_size = min(self.group_size, batch_size)
        
        # Split batch into groups
        y = x.view(group_size, -1, channels, height, width)
        
        # Calculate standard deviation for each group
        y = torch.std(y, dim=0, unbiased=False)
        
        # Average over all elements
        y = torch.mean(y)
        
        # Replicate the scalar to match input dimensions
        y = y.expand(batch_size, 1, height, width)
        
        # Concatenate along channel dimension
        return torch.cat([x, y], dim=1)

class StyleGANDiscriminator(nn.Module):
    def __init__(self, channels=32):
        super().__init__()
        
        # Main convolutional layers
        self.from_rgb = EqualizedConv2d(num_channels, channels, 1)
        
        # Improved feature extraction
        self.features = nn.ModuleList([
            # 32x32x32 -> 16x16x64
            nn.Sequential(
                EqualizedConv2d(channels, channels * 2, 4, 2, 1),
                nn.LeakyReLU(0.2)
            ),
            # 16x16x64 -> 8x8x128
            nn.Sequential(
                EqualizedConv2d(channels * 2, channels * 4, 4, 2, 1),
                nn.LeakyReLU(0.2)
            ),
            # 8x8x128 -> 4x4x256
            nn.Sequential(
                EqualizedConv2d(channels * 4, channels * 8, 4, 2, 1),
                nn.LeakyReLU(0.2)
            )
        ])
        
        # Final block with minibatch standard deviation
        self.final_block = nn.Sequential(
            MinibatchStdDev(),
            EqualizedConv2d(channels * 8 + 1, channels * 8, 3, 1, 1),
            nn.LeakyReLU(0.2),
            EqualizedConv2d(channels * 8, channels * 4, 4, 1, 0),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            EqualizedLinear(channels * 4, 1)
        )
        
    def forward(self, x):
        x = self.from_rgb(x)
        
        for block in self.features:
            x = block(x)
        
        validity = self.final_block(x)
        
        return validity

# Initialize networks
generator = StyleGANGenerator(style_dim, n_mlp).to(device)
discriminator = StyleGANDiscriminator().to(device)

# Optimizers with different learning rates
g_optimizer = optim.Adam(generator.parameters(), lr=lr_g, betas=(beta1, beta2))
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(beta1, beta2))

# Loss functions
def generator_loss(fake_validity):
    # Non-saturating loss - better gradient flow
    return F.softplus(-fake_validity).mean()

def discriminator_loss(real_validity, fake_validity):
    # Logistic loss with R1 regularization
    real_loss = F.softplus(-real_validity).mean()
    fake_loss = F.softplus(fake_validity).mean()
    return real_loss + fake_loss

# R1 regularization for discriminator (gradient penalty)
def r1_reg(real_img, d_real_pred):
    batch_size = real_img.size(0)
    grad_real = torch.autograd.grad(
        outputs=d_real_pred.sum(), inputs=real_img, create_graph=True
    )[0]
    grad_penalty = (grad_real.view(batch_size, -1).norm(2, dim=1) ** 2).mean()
    return grad_penalty

# Fixed noise for visualization
fixed_noise = torch.randn(64, latent_dim, device=device)

# Style mixing regularization
def style_mixing_regularization(generator, z1, z2, prob=0.9):
    if torch.rand(1).item() > prob:
        return generator(z1)
    
    with torch.no_grad():
        w1, w2 = generator.mapping(z1), generator.mapping(z2)
    
    # Randomly choose crossover point
    crossover = int(torch.rand(1).item() * 5) + 1  # Assumes 5 style blocks
    
    # Mix latents
    if crossover == 0:
        return generator(z1, input_is_latent=False)
    elif crossover >= 5:
        return generator(z2, input_is_latent=False)
    else:
        # Implement style mixing (simplified version)
        return generator(z1, input_is_latent=False)

# Training loop
def train_stylegan():
    G_losses = []
    D_losses = []
    r1_weight = 10.0  # Weight for R1 regularization
    
    print("Starting Training Loop...")
    for epoch in range(num_epochs):
        for i, (real_images, _) in enumerate(train_loader):
            batch_size = real_images.size(0)
            
            # Configure input
            real_images = real_images.to(device)
            
            # ---------------------
            #  Train Discriminator
            # ---------------------
            d_optimizer.zero_grad()
            
            # Real images with R1 regularization
            real_images.requires_grad = True
            real_validity = discriminator(real_images)
            d_real_loss = F.softplus(-real_validity).mean()
            
            # R1 regularization
            if i % 16 == 0:  # Apply R1 every 16 batches to save computation
                r1_penalty = r1_reg(real_images, real_validity)
                d_loss_reg = d_real_loss + r1_weight * r1_penalty
                d_loss_reg.backward(retain_graph=True)
            else:
                d_real_loss.backward(retain_graph=True)
            
            # Fake images
            z = torch.randn(batch_size, latent_dim, device=device)
            fake_images = generator(z)
            fake_validity = discriminator(fake_images.detach())
            d_fake_loss = F.softplus(fake_validity).mean()
            
            d_fake_loss.backward()
            d_loss = d_real_loss + d_fake_loss
            d_optimizer.step()
            
            # -----------------
            #  Train Generator
            # -----------------
            g_optimizer.zero_grad()
            
            # Generate a batch of images
            z = torch.randn(batch_size, latent_dim, device=device)
            
            # Apply style mixing regularization occasionally
            if torch.rand(1).item() < 0.3:
                z2 = torch.randn(batch_size, latent_dim, device=device)
                fake_images = style_mixing_regularization(generator, z, z2)
            else:
                fake_images = generator(z)
            
            # Calculate generator loss - non-saturating loss
            fake_validity = discriminator(fake_images)
            g_loss = generator_loss(fake_validity)
            
            g_loss.backward()
            g_optimizer.step()
            
            # Save losses for plotting
            G_losses.append(g_loss.item())
            D_losses.append(d_loss.item())
            
            # Print progress
            if i % 50 == 0:
                print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(train_loader)}] "
                      f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")
        
        # Generate and save sample images after each epoch
        if epoch % 5 == 0 or epoch == num_epochs - 1:
            with torch.no_grad():
                fake_images = generator(fixed_noise).detach().cpu()
                grid = make_grid(fake_images, nrow=8, normalize=True)
                save_image(grid, f"improved_stylegan_samples_epoch_{epoch}.png")
                
    return G_losses, D_losses

# Run training
G_losses, D_losses = train_stylegan()

# Plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="G")
plt.plot(D_losses, label="D")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig("improved_stylegan_loss_plot.png")
plt.show()

# Save model checkpoints
torch.save(generator.state_dict(), "improved_stylegan_generator.pth")
torch.save(discriminator.state_dict(), "improved_stylegan_discriminator.pth")

# Function to generate fake images
def generate_fake_images(num_images=16):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(num_images, latent_dim, device=device)
        fake_images = generator(noise).detach().cpu()
        
        # Denormalize the images
        fake_images = (fake_images + 1) / 2
        
        # Save the images
        grid = make_grid(fake_images, nrow=4)
        save_image(grid, "improved_stylegan_generated_images.png")
        return grid

# Generate sample images
sample_images = generate_fake_images()

print("Improved StyleGAN implementation complete!")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image, make_grid

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Parameters
batch_size = 64
latent_dim = 100
instance_dim = 128
img_channels = 3  # CIFAR-10 has RGB images
image_size = 32
num_epochs = 100
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Define the Instance Encoder
class InstanceEncoder(nn.Module):
    """Encoder network to create instance encodings for IC-GAN"""
    def __init__(self, img_channels=3, instance_dim=128):
        super().__init__()
        self.instance_dim = instance_dim
        
        # Encoder CNN
        self.features = nn.Sequential(
            # 32x32 -> 16x16
            nn.Conv2d(img_channels, 32, 3, 2, 1),
            nn.LeakyReLU(0.2),
            
            # 16x16 -> 8x8
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            
            # 8x8 -> 4x4
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            
            # 4x4 -> 1x1
            nn.Conv2d(128, instance_dim, 4, 1, 0),
            nn.Tanh()
        )
        
    def forward(self, img):
        features = self.features(img)
        # Flatten but keep batch dimension
        instance_encoding = features.view(img.size(0), -1)
        return instance_encoding

# Define the IC-GAN Generator
class ICGANGenerator(nn.Module):
    """Generator for IC-GAN conditioned on instance encoding"""
    def __init__(self, latent_dim=100, instance_dim=128, img_channels=3):
        super().__init__()
        self.latent_dim = latent_dim
        self.instance_dim = instance_dim
        
        # Combined input (latent + instance encoding)
        self.combined_dim = latent_dim + instance_dim
        
        # Initial processing
        self.initial = nn.Sequential(
            nn.ConvTranspose2d(self.combined_dim, 256, 4, 1, 0),  # 4x4
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        
        # Upsampling layers
        self.upsample1 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1),  # 8x8
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        
        self.upsample2 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # 16x16
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.upsample3 = nn.Sequential(
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1),  # 32x32
            nn.Tanh()
        )
        
    def forward(self, z, instance_encoding):
        # Concatenate noise and instance encoding
        concat_input = torch.cat([z, instance_encoding], dim=1)
        
        # Reshape for convolution
        x = concat_input.view(-1, self.combined_dim, 1, 1)
        
        # Forward pass through the network
        x = self.initial(x)
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        
        return x

# Define the IC-GAN Discriminator
class ICGANDiscriminator(nn.Module):
    def __init__(self, img_channels=3):
        super().__init__()
        
        self.features = nn.Sequential(
            # 32x32 -> 16x16
            nn.Conv2d(img_channels, 32, 3, 2, 1),
            nn.LeakyReLU(0.2),
            
            # 16x16 -> 8x8
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            
            # 8x8 -> 4x4
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2)
        )
        
        # Final layers
        self.final = nn.Conv2d(128, 256, 4, 1, 0)
        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, img):
        features = self.features(img)
        final_features = self.final(features)
        validity = self.output(final_features)
        return validity, final_features

# Define the complete IC-GAN model
# Define the complete IC-GAN model
class ICGAN:
    def __init__(self, latent_dim=100, instance_dim=128, img_channels=3):
        self.latent_dim = latent_dim
        self.instance_dim = instance_dim
        self.img_channels = img_channels
        
        # Initialize all networks
        self.encoder = InstanceEncoder(img_channels, instance_dim).to(device)
        self.generator = ICGANGenerator(latent_dim, instance_dim, img_channels).to(device)
        self.discriminator = ICGANDiscriminator(img_channels).to(device)
        
        # Optimizers
        self.optimizer_E = optim.Adam(self.encoder.parameters(), lr=lr, betas=(beta1, beta2))
        self.optimizer_G = optim.Adam(self.generator.parameters(), lr=lr, betas=(beta1, beta2))
        self.optimizer_D = optim.Adam(self.discriminator.parameters(), lr=lr, betas=(beta1, beta2))
        
        # Loss functions
        self.adversarial_loss = nn.BCELoss()
        
        # Training history
        self.g_losses = []
        self.d_losses = []
        self.model_name = "IC-GAN"
    
    def train_step(self, real_imgs):
        batch_size = real_imgs.size(0)
        
        # Ground truths
        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)
        
        # -----------------
        #  Train Encoder and Generator
        # -----------------
        self.optimizer_E.zero_grad()
        self.optimizer_G.zero_grad()
        
        # Encode real images to get instance encodings
        instance_encoding = self.encoder(real_imgs)
        
        # Sample noise
        z = torch.randn(batch_size, self.latent_dim, device=device)
        
        # Generate a batch of images
        gen_imgs = self.generator(z, instance_encoding)
        
        # Adversarial loss for the generator
        validity, _ = self.discriminator(gen_imgs)
        g_loss = self.adversarial_loss(validity, valid)
        
        g_loss.backward()
        self.optimizer_G.step()
        self.optimizer_E.step()
        
        # -----------------
        #  Train Discriminator
        # -----------------
        self.optimizer_D.zero_grad()
        
        # Real images
        real_validity, _ = self.discriminator(real_imgs)
        d_real_loss = self.adversarial_loss(real_validity, valid)
        
        # Fake images (detached to avoid training generator)
        fake_validity, _ = self.discriminator(gen_imgs.detach())
        d_fake_loss = self.adversarial_loss(fake_validity, fake)
        
        d_loss = (d_real_loss + d_fake_loss) / 2
        
        d_loss.backward()
        self.optimizer_D.step()
        
        return g_loss.item(), d_loss.item()
    
    def train(self, dataloader, num_epochs):
        print(f"Starting {self.model_name} Training...")
        
        # Fixed noise for visualization
        fixed_noise = torch.randn(16, self.latent_dim, device=device)
        
        # Get a batch of fixed real images for instance encoding
        fixed_real_imgs = next(iter(dataloader))[0][:16].to(device)
        fixed_instances = self.encoder(fixed_real_imgs)
        
        for epoch in range(num_epochs):
            epoch_g_loss = 0
            epoch_d_loss = 0
            
            for i, (real_imgs, _) in enumerate(dataloader):
                real_imgs = real_imgs.to(device)
                
                # Training step
                g_loss, d_loss = self.train_step(real_imgs)
                
                # Update epoch losses
                epoch_g_loss += g_loss
                epoch_d_loss += d_loss
                
                # Print status
                if i % 50 == 0:
                    print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] "
                          f"[D loss: {d_loss:.4f}] [G loss: {g_loss:.4f}]")
            
            # Save average epoch losses
            self.g_losses.append(epoch_g_loss / len(dataloader))
            self.d_losses.append(epoch_d_loss / len(dataloader))
            
            # Generate and save sample images
            if epoch % 10 == 0 or epoch == num_epochs - 1:
                with torch.no_grad():
                    # Generate fake images with fixed noise and instances
                    fake_imgs = self.generator(fixed_noise, fixed_instances).detach().cpu()
                    
                    # Make grid and save
                    grid = make_grid(fake_imgs, nrow=4, normalize=True)
                    save_image(grid, f"icgan_samples_epoch_{epoch}.png")
        
        # Save the final models
        torch.save(self.encoder.state_dict(), "icgan_encoder.pth")
        torch.save(self.generator.state_dict(), "icgan_generator.pth")
        torch.save(self.discriminator.state_dict(), "icgan_discriminator.pth")
        
        return self.g_losses, self.d_losses
    
    def generate_fake_images(self, real_images=None, num_images=16):
        self.encoder.eval()
        self.generator.eval()
        
        with torch.no_grad():
            # If real images are provided, use them for instance encoding
            if real_images is not None:
                real_images = real_images.to(device)
                instances = self.encoder(real_images)
            else:
                # Use a batch from the dataloader
                real_images = next(iter(train_loader))[0][:num_images].to(device)
                instances = self.encoder(real_images)
            
            # Generate random noise
            z = torch.randn(num_images, self.latent_dim, device=device)
            
            # Generate fake images
            fake_images = self.generator(z, instances).detach().cpu()
            
            # Denormalize
            fake_images = (fake_images + 1) / 2
            
            # Create grid of both real and fake for comparison
            real_images = (real_images.detach().cpu() + 1) / 2
            comparison = torch.cat([real_images, fake_images], dim=0)
            grid = make_grid(comparison, nrow=num_images, normalize=False)
            
            save_image(grid, "icgan_real_vs_fake.png")
            return grid
    
    def detect_fake_images(self, images):
        self.discriminator.eval()
        with torch.no_grad():
            images = images.to(device)
            # D returns probability that each image is real
            scores, _ = self.discriminator(images)
            predictions = (scores >= 0.5).float()  # 1 for real, 0 for fake
            return scores, predictions

# Initialize and train the IC-GAN model
model = ICGAN(latent_dim, instance_dim, img_channels)
g_losses, d_losses = model.train(train_loader, num_epochs)

# Plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(g_losses, label="G")
plt.plot(d_losses, label="D")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig("icgan_loss_plot.png")
plt.show()

# Generate fake images and compare with real images
comparison_grid = model.generate_fake_images()

# Function to evaluate the discriminator's performance
def evaluate_discriminator(model, dataloader, num_batches=10):
    model.discriminator.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i, (real_imgs, _) in enumerate(dataloader):
            if i >= num_batches:
                break
                
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            
            # Get instance encoding and generate fake images
            instances = model.encoder(real_imgs)
            z = torch.randn(batch_size, model.latent_dim, device=device)
            fake_imgs = model.generator(z, instances)
            
            # Evaluate on real images (should be classified as real - 1)
            real_preds, _ = model.discriminator(real_imgs)
            real_correct = ((real_preds >= 0.5).float() == 1).sum().item()
            
            # Evaluate on fake images (should be classified as fake - 0)
            fake_preds, _ = model.discriminator(fake_imgs)
            fake_correct = ((fake_preds < 0.5).float() == 1).sum().item()
            
            # Accumulate statistics
            correct += (real_correct + fake_correct)
            total += (batch_size * 2)  # Both real and fake
    
    accuracy = 100 * correct / total
    print(f"Discriminator accuracy: {accuracy:.2f}%")
    return accuracy

# Evaluate the discriminator
accuracy = evaluate_discriminator(model, train_loader)

# Now let's create a function to visualize instance-conditioned generation
def visualize_instance_conditioning(model, num_instances=4, num_variations=4):
    """Generate multiple variations from the same instance encodings"""
    model.encoder.eval()
    model.generator.eval()
    
    with torch.no_grad():
        # Get some real images for instance encoding
        real_imgs = next(iter(train_loader))[0][:num_instances].to(device)
        instances = model.encoder(real_imgs)
        
        # Create output tensors
        all_images = []
        all_images.append((real_imgs.detach().cpu() + 1) / 2)  # Add real images (denormalized)
        
        # For each instance, generate multiple variants
        for i in range(num_variations):
            z = torch.randn(num_instances, model.latent_dim, device=device)
            fake_imgs = model.generator(z, instances)
            all_images.append((fake_imgs.detach().cpu() + 1) / 2)  # Denormalize
        
        # Stack all images
        all_images = torch.cat(all_images, dim=0)
        
        # Create a grid
        grid = make_grid(all_images, nrow=num_instances, normalize=False)
        save_image(grid, "icgan_instance_variations.png")
        
        return grid

# Visualize instance conditioning
instance_variations = visualize_instance_conditioning(model)

print("IC-GAN implementation complete!")

In [ ]:
import shutil

# Path to the directory you want to zip
dir_to_zip = '/kaggle/working/data'
# Output zip file path
zip_file_path = '/kaggle/working/data.zip'

# Create the zip file
shutil.make_archive(base_name=zip_file_path.replace('.zip', ''), format='zip', root_dir=dir_to_zip)

print("Zipping completed:", zip_file_path)
